In [1]:
!pip install opendartreader

nginx service is running...
sshd service is running...
redis-server service is running...


In [2]:
api_key = '24c24e732043f024a09a8ca99d75bcea19837bdf'

import OpenDartReader
import time
from datetime import datetime

dart = OpenDartReader(api_key)

def get_balance_sheet_year(symbol, year):
    '''
    reprt_code 항목
        11013  : 1분기
        11012  : 2분기
        11014  : 3분기
        11011  : 4분기
    '''
    
    report = dart.finstate(
        corp = symbol,
        bsns_year = year,
        reprt_code = '11011'
    )
    '''
        rcept_no: 접수번호
        corp_code: 사업 연도
        stock_code: 종목 코드
        reprt_code: 보고서 코드
        account_nm: 계정명 (예: 자본총계)
        fs_div: 개별/연결구분 ('CFS'=연결재무제표, 'OFS'=재무제표)
        fs_nm: 개별/연결명 ('연결재무제표' 또는 '재무제표')
        sj_div: 재무제표구분 ('BS'=재무상태표, 'IS'=손익계산서)
        sj_nm: 재무제표명 ( '재무상태표' 또는 '손익계산서')
        thstrm_nm: 당기명
        thstrm_dt: 당기일자
        thstrm_amount: 당기금액
        thstrm_add_amount: 당기누적금액
        frmtrm_nm: 전기명
        frmtrm_dt: 전기일자
        frmtrm_amount: 전기금액
        frmtrm_add_amount: 전기누적금액
        bfefrmtrm_nm: 전전기명
        bfefrmtrm_dt: 전전일자
        bfefrmtrm_amount: 전전기금액
        ord: 계정과목 정렬순서
    '''
    report = report.rename(columns={'account_nm': '계정명', 'thstrm_amount': '당기금액', 'frmtrm_amount': '전기금액', 'bfefrmtrm_amount': '전전기금액'})
    # report.columns = ['접수번호', '사업 연도', '종목 코드', '보고서 코드', '계정명', '개별/연결구분', '개별/연결명', '재무제표구분', '재무제표명', '당기명', '당기일자', '당기금액', '당기누적금액', '전기명', '전기일자', '전기금액', '전기누적금액', '전전기명', '전전일자', '전전기금액', '계정과목 정렬순서']
    print(report)
    pass
    return report

def get_balance_sheet_quarter(symbol):
    
    column_name = {'rcept_no': '접수번호',
                    'corp_code': '고유번호',
                    'bsns_year': '사업연도',
                    'stock_code': '종목 코드',
                    'reprt_code': '보고서 코드',
                    'account_nm': '계정명', # (예: 자본총계)'
                    'fs_div': '개별연결구분', # (CFS=연결재무제표, OFS=재무제표)'
                    'fs_nm': '개별연결명', # (연결재무제표 또는 재무제표)'
                    'sj_div': '재무제표구분', # (BS=재무상태표, IS=손익계산서)'
                    'sj_nm': '재무제표명', # (재무상태표 또는 손익계산서)'
                    'thstrm_nm': '당기명',
                    'thstrm_dt': '당기일자',
                    'thstrm_amount': '당기금액',
                    'thstrm_add_amount': '당기누적금액',
                    'frmtrm_nm': '전기명',
                    'frmtrm_dt': '전기일자',
                    'frmtrm_amount': '전기금액',
                    'frmtrm_add_amount': '전기누적금액',
                    'bfefrmtrm_nm': '전전기명',
                    'bfefrmtrm_dt': '전전일자',
                    'bfefrmtrm_amount': '전전기금액',
                    'ord': '계정과목 정렬순서',}
    
    quarters = ['11013', '11012', '11014', '11011'] # 1분기, 2분기, 3분기, 4분기
    _months = [3, 6, 9, 12]
    current_year = datetime.now().year
    current_month = datetime.now().month
    # current_month = 9
    
    
    current_quarter = (current_month-1)//3
    quarter = 0
    year = 0 
    report = None
    for i in range(2):
        # print('--------------'*5,'\ni:', i)
        i = i + 1
        # print(current_quarter)
        quarter = ((current_month-1)//3 - i)%4
        year = current_year + (current_quarter  - i )//4
        # print('quarter:', current_quarter, quarter, quarters[quarter], year)
        # print('quaters:', quarters)
        # print('quater:', current_quarter, quarter, year)
        print('symbol:', symbol, 'year:', year, 'quarter:', quarters[quarter])
        report = dart.finstate(symbol, year, quarters[quarter])
        # print(report.columns)
        report = report.rename(columns=column_name)
        # print(report.columns)
        is_empty = is_empty_report(report)
        if not is_empty:
            continue
        # report = report.rename(columns={
        #     'rcept_no': '접수번호', 
        #     'corp_code': '사업 연도',
        #     'stock_code': '종목 코드',
        #     'reprt_code': '보고서 코드',
        #     'account_nm': '계정명',    # (예: 자본총계)',
        #     'fs_div': '개별/연결구분',
        #     'fs_nm': '개별/연결명',
        #     'sj_div': '재무제표구분',   # ('BS'=재무상태표, 'IS'=손익계산서)
        #     'sj_nm': '재무제표명',      # ( '재무상태표' 또는 '손익계산서')
        #     'thstrm_nm': '당기명',
        #     'thstrm_dt': '당기일자',
        #     'thstrm_amount': '당기금액',
        #     'thstrm_add_amount': '당기누적금액',
        #     'frmtrm_nm': '전기명',
        #     'frmtrm_dt': '전기일자',
        #     'frmtrm_amount': '전기금액',
        #     'frmtrm_add_amount': '전기누적금액',
        #     'bfefrmtrm_nm': '전전기명',
        #     'bfefrmtrm_dt': '전전일자',
        #     'bfefrmtrm_amount': '전전기금액',
        #     'ord': '계정과목 정렬순서'
        #     })
        time.sleep(0.07)
        
    print(symbol, year, quarters[quarter])
    
    
    return report
    
    # year = current_year - (quarter+1)//4
    # reprt_code = quarters[quarter]
    
    
    # print('year:', year, 'quarter:', quarter)
    
    # is_empty = is_empty_report(dart.finstate(symbol, year, reprt_code))
    
    
    # year_reprt_code = []
    # if is_empty:
    #     reprt_code = quarters[quarter-1]
    #     year = year - 
    # else:
    #     year_reprt_code.append(dart.finstate(symbol, year, reprt_code))
    # pass



def is_empty_report(report):
    if report.empty:
        return True
    else:
        return False

In [3]:
# report = get_balance_sheet_year('207940', 2023)
# print(report["계정명"])
report = get_balance_sheet_quarter('207940')
print(report.columns)
# report = report.rename(columns={
#             'rcept_no': '접수번호', 
#             'corp_code': '사업 연도',
#             'stock_code': '종목 코드',
#             'reprt_code': '보고서 코드',
#             'account_nm': '계정명',    # (예: 자본총계)',
#             'fs_div': '개별/연결구분',
#             'fs_nm': '개별/연결명',
#             'sj_div': '재무제표구분',   # ('BS'=재무상태표, 'IS'=손익계산서)
#             'sj_nm': '재무제표명',      # ( '재무상태표' 또는 '손익계산서')
#             'thstrm_nm': '당기명',
#             'thstrm_dt': '당기일자',
#             'thstrm_amount': '당기금액',
#             'thstrm_add_amount': '당기누적금액',
#             'frmtrm_nm': '전기명',
#             'frmtrm_dt': '전기일자',
#             'frmtrm_amount': '전기금액',
#             'frmtrm_add_amount': '전기누적금액',
#             'bfefrmtrm_nm': '전전기명',
#             'bfefrmtrm_dt': '전전일자',
#             'bfefrmtrm_amount': '전전기금액',
#             'ord': '계정과목 정렬순서'
#             })

symbol: 207940 year: 2024 quarter: 11011
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

symbol: 207940 year: 2024 quarter: 11014
207940 2024 11014
Index(['접수번호', '보고서 코드', '사업연도', '고유번호', '종목 코드', '개별연결구분', '개별연결명', '재무제표구분',
       '재무제표명', '계정명', '당기명', '당기일자', '당기금액', '전기명', '전기일자', '전기금액',
       '계정과목 정렬순서', 'currency', '당기누적금액', '전기누적금액'],
      dtype='object')


In [4]:
data = dart.finstate('207940', '2024', '11014')

In [5]:
report  = dart.list('207940', start='2022', kind='A', final=False)
for ix, row in report.iterrows():
    print(row['rcept_dt'], 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + row['rcept_no'],  row['report_nm'])
sub_report = dart.sub_docs('20241114002286')
print(sub_report)






20241114 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20241114002286 분기보고서 (2024.09)
20240814 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20240814003571 반기보고서 (2024.06)
20240516 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20240516001936 분기보고서 (2024.03)
20240307 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20240307000835 사업보고서 (2023.12)
20231114 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20231114002501 분기보고서 (2023.09)
20230814 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20230814002651 반기보고서 (2023.06)
20230515 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20230515002481 분기보고서 (2023.03)
20230309 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20230309000835 사업보고서 (2022.12)
20221114 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20221114002081 분기보고서 (2022.09)
20220816 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220816002318 반기보고서 (2022.06)
20220516 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220516002080 분기보고서 (2022.03)
20220321 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220321000862 사업보고서 (2